# Transfer Learning with Pytorch

This notebook showcases how to perform transfer learning with pytorch.
Two sections are provided:
1. Training: pre-trained DenseNet121 model is loaded and trained with a reduced Cats & Dogs dataset after re-organizing it.
2. Inference: saved trained model is used to perform inference.

## 1. Training